In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import os
import glob
import time
import pandas as pd

# ==============================================================================
# 1. 設定・パラメータ定義エリア
#    ここでの設定値は、物理的な定義や事前の計測に基づいています。
# ==============================================================================

# 学習済みモデルのパス
MODEL_PATH = 'runs/detect/train9/weights/best.pt'
# 検証用動画が入っているフォルダ
VIDEO_DIR = 'video'
# 正解ラベルが書かれたCSVファイル
CSV_PATH = 'Label.csv' 

# --- ストライクゾーン構築のためのパラメータ ---
# 根拠: 高校球児の平均身長(約171cm)とホームベース幅(43.2cm)の比率から算出
# これにより、カメラの距離が変わっても適切なゾーンを自動生成できます。

ZONE_W_RATIO = 1.0         # ゾーン横幅 (ベース幅と同じ)
ZONE_H_RATIO = 1.09        # ゾーン高さ (身長の胸〜膝の長さをベース幅で割った値)
ZONE_OFFSET_RATIO = 1.03   # ゾーン下限の高さ (地面から膝下までの高さをベース幅で割った値)

# --- 判定ロジックのためのパラメータ ---
BALL_SIZE_RATIO = 0.18     # 奥行き判定: ボールがベース幅の「0.18倍」になったら通過とみなす
IGNORE_TOP_RATIO = 0.2     # 誤検知対策: 画面上部20%（天井など）は無視する
STRICT_MODE = False        # 判定モード: Falseなら「ボールが少しかすってもストライク」とする

# ==============================================================================
# 2. データ読み込み用関数
#    CSVの表記ゆれ（1.0, 1, 1.mp4など）を吸収して読み込む処理
# ==============================================================================
def load_labels(csv_path):
    """
    正解ラベルCSVを読み込み、辞書形式 {ファイルID: 正解ラベル} で返す。
    """
    print(f"CSV読み込み中: {csv_path}")
    if not os.path.exists(csv_path):
        print("【エラー】CSVファイルが見つかりません。")
        return {}
    
    try:
        df = pd.read_csv(csv_path)
        answers = {}
        
        for index, row in df.iterrows():
            raw_id = row.iloc[0]
            label_raw = str(row.iloc[1]).strip().upper()
            
            # IDの表記ゆれを修正 (例: 1.0 -> 1)
            try:
                clean_id = str(int(float(raw_id)))
            except:
                clean_id = str(raw_id).replace('.mp4', '').strip()

            # ラベルの表記ゆれを修正
            if "ストライク" in label_raw: label = "STRIKE"
            elif "ボール" in label_raw: label = "BALL"
            else: label = label_raw
            
            answers[clean_id] = label
            
        print(f"正解データを {len(answers)} 件読み込みました。")
        return answers
        
    except Exception as e:
        print(f"【エラー】CSV読み込み中にエラーが発生しました: {e}")
        return {}

# ==============================================================================
# 3. 判定クラス (BatchJudge)
#    1つの動画を受け取り、AI解析を行って結果を返すクラス
# ==============================================================================
class BatchJudge:
    def __init__(self, model):
        self.model = model
        # ゾーン計算を安定させるための変数
        self.plate_width_history = []
        self.fixed_plate_width = 0
        self.zone_coords = None
        
        # ボールの動きを追うための変数
        self.prev_ball = None
        
        # 最終結果 (初期値は不明)
        self.final_judgment = "UNKNOWN"

    def predict(self, video_path):
        """
        動画を1フレームずつ読み込み、ストライク/ボールを判定する
        """
        cap = cv2.VideoCapture(video_path)
        
        # 動画のサイズを取得
        w_org = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        h_org = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # スマホ撮影など、縦横が逆転している場合の自動回転判定
        if w_org > h_org:
            rotate = True
            screen_h = w_org # 回転後は幅が高さになる
        else:
            rotate = False
            screen_h = h_org

        # 天井照明などの誤検知を防ぐための無視ライン
        ignore_y = int(screen_h * IGNORE_TOP_RATIO)
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break
            
            # 回転が必要なら90度回転させる
            if rotate: frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

            # YOLOv8による物体検出とトラッキング
            # persist=True にすることで、前後のフレームの同一物体をID管理する
            results = self.model.track(frame, persist=True, conf=0.15, verbose=False)
            
            curr_ball = None
            curr_plate = None

            # 検出結果の解析
            if results[0].boxes.id is not None:
                boxes = results[0].boxes.xywh.cpu().numpy()
                classes = results[0].boxes.cls.cpu().numpy()
                
                for box, cls_id in zip(boxes, classes):
                    bx, by, bw, bh = box
                    
                    # 画面上部（天井）の検出は無視する
                    if by < ignore_y: continue 

                    # クラス0: ボール
                    if int(cls_id) == 0:
                        # 複数検出された場合は、最も大きい（手前にある）ボールを採用
                        if curr_ball is None or bw > curr_ball[2]:
                            curr_ball = box
                            
                    # クラス1: ホームベース
                    elif int(cls_id) == 1:
                        if curr_plate is None or bw > curr_plate[2]:
                            curr_plate = box

            # --- A. ストライクゾーンの動的生成 ---
            # ホームベースが検出されていれば、その幅を基準にゾーンを計算する
            if curr_plate is not None:
                px, py, pw, ph = curr_plate
                
                # 幅の数値を安定させるために移動平均をとる
                self.plate_width_history.append(pw)
                if len(self.plate_width_history) > 10: self.plate_width_history.pop(0)
                self.fixed_plate_width = sum(self.plate_width_history) / len(self.plate_width_history)

                # ベース幅を「定規」としてゾーン座標を計算
                base_w = self.fixed_plate_width
                zw = int(base_w * ZONE_W_RATIO)
                zh = int(zw * ZONE_H_RATIO)
                zb = int(py - (base_w * ZONE_OFFSET_RATIO)) # 下限
                zt = zb - zh                                # 上限
                zx = int(px)
                zl, zr = zx - zw//2, zx + zw//2
                
                self.zone_coords = (zl, zt, zr, zb)

            # --- B. 投球判定ロジック ---
            # まだ判定が出ておらず、ゾーンが確定している場合に実行
            if self.final_judgment == "UNKNOWN" and self.zone_coords and self.fixed_plate_width > 0:
                
                # 判定すべきボールのサイズ（通過基準）を計算
                target_w = self.fixed_plate_width * BALL_SIZE_RATIO
                
                if curr_ball is not None:
                    bx, by, bw, bh = curr_ball
                    
                    # ボールの大きさが基準を超えたら判定を行う
                    if bw >= target_w:
                        ball_radius = bw / 2
                        judge_result = None
                        
                        # 【重要】補間処理
                        # フレーム間でボールが急に大きくなった場合、
                        # ちょうど基準サイズになった瞬間の座標を計算して精度を高める
                        if self.prev_ball is not None:
                            prev_x, prev_y, prev_w, prev_h = self.prev_ball
                            if prev_w < target_w:
                                # 線形補間により通過座標を推定
                                ratio = (target_w - prev_w) / (bw - prev_w)
                                cross_y = prev_y + (by - prev_y) * ratio
                                cross_x = prev_x + (bx - prev_x) * ratio
                                judge_result = self._check_zone(cross_x, cross_y, ball_radius)
                            else:
                                judge_result = self._check_zone(bx, by, ball_radius)
                        else:
                            judge_result = self._check_zone(bx, by, ball_radius)
                        
                        # 判定が出たらループを抜ける（処理高速化のため）
                        if judge_result:
                            self.final_judgment = judge_result
                            break

                    self.prev_ball = curr_ball

        cap.release()
        return self.final_judgment

    def _check_zone(self, x, y, radius):
        """
        座標(x,y)がストライクゾーンに入っているか判定する
        """
        zl, zt, zr, zb = self.zone_coords
        
        # 判定条件
        # STRICT_MODEがFalseの場合、ボールの端がかかっていればOKとする
        if STRICT_MODE:
            is_y_in = (zt <= y <= zb)
            is_x_in = (zl <= x <= zr)
        else:
            is_y_in = (y + radius >= zt) and (y - radius <= zb)
            is_x_in = (x + radius >= zl) and (x - radius <= zr)

        if is_y_in and is_x_in:
            return "STRIKE"
        else:
            return "BALL"

# ==============================================================================
# 4. メイン実行ブロック
#    全動画ファイルを順次読み込み、精度を検証する
# ==============================================================================

# モデルの読み込み
print(f"モデル読み込み中...: {MODEL_PATH}")
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("モデルが見つかりません")
shared_model = YOLO(MODEL_PATH)

# 正解データの読み込み
CORRECT_ANSWERS = load_labels(CSV_PATH)

# 動画ファイルリストの取得（ファイル名順にソート）
video_files = glob.glob(os.path.join(VIDEO_DIR, "*.mp4"))
video_files.sort(key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

print(f"対象動画数: {len(video_files)}本")
print("-" * 60)
print(f"{'File':<6} | {'AI Pred':<10} | {'Correct':<10} | {'Result'}")
print("-" * 60)

correct_count = 0
total_checked = 0
unknown_count = 0

start_time = time.time()

# 全動画ループ
for video_path in video_files:
    file_id = os.path.splitext(os.path.basename(video_path))[0]
    
    # AIによる判定
    judge = BatchJudge(shared_model)
    ai_result = judge.predict(video_path)
    
    # 正解ラベルの取得
    correct_label = CORRECT_ANSWERS.get(file_id, "---")
    
    status = ""
    if correct_label != "---":
        total_checked += 1
        if ai_result == correct_label:
            status = "OK"
            correct_count += 1
        else:
            status = "NG"
    
    if ai_result == "UNKNOWN":
        unknown_count += 1

    # 結果の表示
    print(f"{file_id:<6} | {ai_result:<10} | {correct_label:<10} | {status}")

elapsed_time = time.time() - start_time

# --- 最終結果レポート ---
print("-" * 60)
print(f"処理完了 ({elapsed_time:.1f}秒)")
print(f"判定不能(UNKNOWN): {unknown_count}本")

if total_checked > 0:
    accuracy = (correct_count / total_checked) * 100
    print(f"\n【最終成績】")
    print(f"正解数: {correct_count} / {total_checked}")
    print(f"正答率: {accuracy:.1f}%")
else:
    print("\n※正解データが見つかりませんでした。")

モデル読み込み中...: runs/detect/train9/weights/best.pt
CSV読み込み中: Label.csv
正解データを 150 件読み込みました。
対象動画数: 150本
------------------------------------------------------------
File   | AI Pred    | Correct    | Result
------------------------------------------------------------
1      | STRIKE     | STRIKE     | OK
2      | STRIKE     | STRIKE     | OK
3      | BALL       | BALL       | OK


KeyboardInterrupt: 